## How to categorize text

Written text is one of humankind's greatest inventions. It enables us to interact with other people across time and space. You can write down your thoughts and a hundred years later another human can pick up your scribblings and read what you wrote. But how will that other human interpret the text? Will they consider it a news article describing some interesting event that occured? Will they consider it a fictional story, a personal letter, or even a divinely inspired scripture? 

This project aims to build a supervised feed-forward neural network that can do the interpretation for us. A basic feed-forward neural network takes in some input, processes it in the so-called hidden layer, and produces an output. In a supervised setting the correctness of the output will then be checked and if it is not the desired one, adjustments will be made to help the algorithm achieve better results.

### Data
As our data we'll use from the NLTK library the Brown corpus which contains documents from 15 different categories. The documents will be further chopped up into 100 word snippets. For the sake of uniformity this means that some data will be discarded if and when a document is not divisible by hundred. The snippets will then be randomly shuffled before dividing them into sets: 20% for the test set, and 80% for the training set. 

In [ ]:
import random
import math
import nltk
from nltk.corpus import brown

documents = [(list(brown.words(fileid)), category) 
             for category in brown.categories() for fileid in brown.fileids(category)]

In [ ]:
def get_snippets(document_list):
    snippets = []
    for text_and_category in document_list:
        text = text_and_category[0]
        category = text_and_category[1]
        lower_limit = 0
        upper_limit = 100
        while upper_limit < len(text):
            snippet = text[lower_limit:upper_limit]
            snippets.append((snippet, category))
            lower_limit += 100
            upper_limit += 100
    return snippets

snippets = get_snippets(documents)
random.shuffle(snippets)
size = int(len(snippets)*0.2)
test_snippets = snippets[:size]
train_snippets = snippets[size:]

### Methods

Given a piece of text, how would a human decide what type of text it is? While a poem might be easy to recognise (if it rhymes), how does one differentiate science fiction from romance fiction? The answer is in the words. We would see words like 'starship' or 'time machine' and based on previous knowledge of the likelihood of these words appearing in a science fiction novel versus the likelihood of the same words appearing in a romance novel, we would choose the more likely option. Although things aren't always this clear cut (what is there preventing a text being science fiction and romance?), it is a good starting point. Additionally, the most commonly used words such as 'the' and 'you' appear in virtually every text, so to lessen the workload, we can exclude those words from our considerations. 

In [ ]:
all_words = nltk.FreqDist(w.lower() for w in brown.words())
most_common = [w for w, f in all_words.most_common(350)]

There already exists a machine learnign algorithm that can classify text into different categories based on prior probabilities: the Naive Bayes Classifier. 

In [ ]:
def get_naive_features(text_snippet):
    featureset = {}
    words = set(text_snippet[0])
    for word in words:
        if word.lower() not in most_common:
            featureset['contains({})'.format(word.lower())] = (word in words)
    return featureset

featuresets = [(get_naive_features(d), c) for (d, c) in snippets]
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print("Accuracy of the Naive Bayes Classifier: {:.2f}%".format((nltk.classify.accuracy(
    classifier, test_set)*100)))

However, the Naive Bayes Classifier that NLTK offers works very poorly for this task (only around 16% accuracy rate). This is because while the Naive Bayes Classifier does look at the words of the text snippet, the words themselves don't carry any weight: it is simply a yes or no, and this doesn't translate very well to deciding between 15 different categories. The solution? We will create our own supervised machine learning algorithm. Let's start by further dividing the training set into separate sets for each category.

In [ ]:
train_learn = []
train_bel = []
train_lore = []
train_news = []
train_hobby = []
train_gov = []
train_rom = []
train_adv = []
train_fic = []
train_edit = []
train_mys = []
train_rev = []
train_rel = []
train_humor = []
train_scifi = []

for snippet in train_snippets:
    if snippet[1] == 'learned':
        train_learn.append(snippet)
    elif snippet[1] == 'belles_lettres':
        train_bel.append(snippet)
    elif snippet[1] == 'lore':
        train_lore.append(snippet)
    elif snippet[1] == 'news':
        train_news.append(snippet)
    elif snippet[1] == 'hobbies':
        train_hobby.append(snippet)
    elif snippet[1] == 'government':
        train_gov.append(snippet)
    elif snippet[1] == 'romance':
        train_rom.append(snippet)
    elif snippet[1] == 'adventure':
        train_adv.append(snippet)    
    elif snippet[1] == 'fiction':
        train_fic.append(snippet)
    elif snippet[1] == 'editorial':
        train_edit.append(snippet)
    elif snippet[1] == 'mystery':
        train_mys.append(snippet)
    elif snippet[1] == 'reviews':
        train_rev.append(snippet)
    elif snippet[1] == 'religion':
        train_rel.append(snippet)
    elif snippet[1] == 'humor':
        train_humor.append(snippet)
    elif snippet[1] == 'science_fiction':
        train_scifi.append(snippet)

As established earlier, the key to categorizing text based on the words that appear in it lies with the prior probabilities. So how do we get them? A simple way to go about it is to count all the words that appear in a certain category (excluding the most common ones) and keep the most frequent words and their frequency as the relevant features. The units in the hidden layer of our neural network will use these frequencies in their computations.

In [ ]:
def get_features(text_snippet, feature_set):
    words = set(text_snippet[0])
    for word in words:
        if word.lower() not in most_common:
            if word.lower() in feature_set:
                feature_set[word.lower()] += 1 
            else:
                feature_set[word.lower()] = 1 

def get_value(pair):
    return pair[1]                


def get_feature_set(train_subset):
    features = {}
    for snippet in train_subset:
        get_features(snippet, features)
    feature_set = {}
    for key, value in sorted(features.items(), key=get_value, reverse=True)[:400]:
        feature_set[key] = value
    return feature_set

learn_features = get_feature_set(train_learn)       
bel_features = get_feature_set(train_bel)
lore_features = get_feature_set(train_lore)
news_features = get_feature_set(train_news)
hobby_features = get_feature_set(train_hobby)
gov_features = get_feature_set(train_gov)
rom_features = get_feature_set(train_rom)
adv_features = get_feature_set(train_adv)
fic_features = get_feature_set(train_fic)
edit_features = get_feature_set(train_edit)
mys_features = get_feature_set(train_mys)
rev_features = get_feature_set(train_rev)
rel_features = get_feature_set(train_rel)
humor_features = get_feature_set(train_humor)
scifi_features = get_feature_set(train_scifi)

The number of snippets in a given category varies greatly. Since the word frequencies will be used to make a decision about the category, this puts the least frequent categories at a disadvantage because their word frequencies will always be much smaller. To correct this, we'll normalize the frequencies so that each category 'has appeared' in the training data roughly the same amount of times.

In [ ]:
max_frequency = max(len(train_learn), len(train_bel), len(train_lore), len(train_news), len(train_hobby), \
                    len(train_gov), len(train_rom), len(train_adv), len(train_fic), len(train_edit), \
                    len(train_mys), len(train_rev), len(train_rel), len(train_humor), len(train_scifi))

def normalize(train_subset, features):
    factor = max_frequency/len(train_subset)
    if factor > 1.5:
        factor = math.ceil(factor)
    else:
        factor = math.floor(factor)
    for key, value in features.items():
        new_value = value*factor
        features[key] = new_value
    return features
        

learn_features = normalize(train_learn, learn_features)
bel_features = normalize(train_bel, bel_features)
lore_features = normalize(train_lore, lore_features)
news_features = normalize(train_news, news_features)
hobby_features = normalize(train_hobby, hobby_features)
gov_features = normalize(train_gov, gov_features)
rom_features = normalize(train_rom, rom_features)
adv_features = normalize(train_adv, adv_features)
fic_features = normalize(train_fic, fic_features)
edit_features = normalize(train_edit, edit_features)
mys_features = normalize(train_mys, mys_features)
rev_features = normalize(train_rev, rev_features)
rel_features = normalize(train_rel, rel_features)
humor_features = normalize(train_humor, humor_features)
scifi_features = normalize(train_scifi, scifi_features)

Now we have the prior probabilities so it is time to set up the hidden layer of our neural network. There will be 15 units in the hidden layer, each of them counting a comparison number for a category. 

In [ ]:
def count(snippet, features):
    total = 0
    for word in snippet:
        if word.lower() in features:
            total += features[word.lower()]
    return total

The output unit of our neural network takes in all the comparison numbers from the hidden layer and simply picks the one that is highest. It then outputs the name of the corresponding category.

In [ ]:
def get_prediction(words):
    options = {}
    options['learned'] = count(words, learn_features)
    options['belles_lettres'] = count(words, bel_features)
    options['lore'] = count(words, lore_features)
    options['news'] = count(words, news_features)
    options['hobbies'] = count(words, hobby_features)
    options['government'] = count(words, gov_features)
    options['romance'] = count(words, rom_features)
    options['adventure'] = count(words, adv_features)
    options['fiction'] = count(words, fic_features)
    options['editorial'] = count(words, edit_features)
    options['mystery'] = count(words, mys_features)
    options['reviews'] = count(words, rev_features)
    options['religion'] = count(words, rel_features)
    options['humor'] = count(words, humor_features)
    options['science_fiction'] = count(words, scifi_features)
    high_score = 0
    prediction = ''
    for key, value in options.items():
        if value > high_score:
            high_score = value
            prediction = key
    return prediction

Because this is a feed-forward network, no information will be passed back to the units, and therefore it cannot improve its performance on its own. However, since we already are supervising the learning process, we can help out by adjusting the features, i.e. word frequencies, whenever our neural network makes a mistake.

In [ ]:
def add_weight(snippet, features):
    for word in snippet:
        if word.lower() in features:
            features[word.lower()] += 1
            
            
def adjust_features(snippet):
    words = snippet[0]
    category = snippet[1]
    prediction = get_prediction(words)
    if prediction != category:
        if category == 'learned':
            add_weight(words, learn_features)
        elif category == 'belles_lettres':
            add_weight(words, bel_features)
        elif category == 'lore':
            add_weight(words, lore_features)
        elif category == 'news':
            add_weight(words, news_features)
        elif category == 'hobbies':
            add_weight(words, hobby_features)
        elif category == 'government':
            add_weight(words, gov_features)
        elif category == 'romance':
            add_weight(words, rom_features)
        elif category == 'adventure':
            add_weight(words, adv_features)
        elif category == 'fiction':
            add_weight(words, fic_features)
        elif category == 'editorial':
            add_weight(words, edit_features)
        elif category == 'mystery':
            add_weight(words, mys_features)
        elif category == 'reviews':
            add_weight(words, rev_features)
        elif category == 'religion':
            add_weight(words, rel_features)
        elif category == 'humor':
            add_weight(words, humor_features)
        elif category == 'science_fiction':
            add_weight(words, scifi_features)

Now we have set up everything so it's time for the network to start the learning process. The network will go through all the text snippets in the training data set and we'll adjust the features whenever a mistake is made. This process, called an epoch, will be repeated twenty times. As this takes some time, the network will keep us informed of it's progress every five epochs. 

In [ ]:
print('Training started.')
epoch = 0
while epoch < 20:
    for snippet in train_snippets:
        adjust_features(snippet)
    if (epoch+1) % 5 == 0:
        print('Finished epoch {}/20.'.format(epoch+1))
    random.shuffle(train_snippets)
    epoch += 1
print('Training finished.')

### Evaluation

Our neural network has finished its training and it's time to see how well it performs on the test data that it has never seen before. The test results will be presented as a confusion matrix: the rows represent the correct category and the columns represent the category that our network picked. The diagonal line from the upper left corner to the bottom right corner contains all the instances where our network got the category right. Due to spacing issues the confusion matrix will not show percentages but the number of snippets that it classified as X. However, we will display the overall accuracy under the confusion matrix along with the accuracy achieved with the NLTK Naive Bayes Classifier, the accuracy with the naive baseline (that is, caegorizing everything as the most frequent category), and the accuracy achieved by simply guessing. 

In [ ]:
test = []
for snippet in test_snippets:
    text = snippet[0]
    correct_category = snippet[1]
    predicted_category = get_prediction(text)
    test.append(predicted_category)
gold = [category for (text, category) in test_snippets]  
cm = nltk.ConfusionMatrix(gold, test)
print(cm.pretty_format(sort_by_count=True))

print('The number of snippets in the test set: {}'.format(len(test_snippets)))
categories_by_frequency = nltk.FreqDist(category for (text, category) in snippets)
print(categories_by_frequency.most_common(), '\n')
accuracy = 0
for i in range(len(gold)):
    if gold[i] == test[i]:
        accuracy += 1       
print('Accuracy of our neural network: {:.2f}%'.format((accuracy/len(gold))*100))

naive = categories_by_frequency.most_common()[0][0]
naive_test = []
while len(naive_test) != len(gold):
    naive_test.append(naive)
naive_accuracy = 0
for i in range(len(gold)):
    if gold[i] == naive_test[i]:
        naive_accuracy += 1       
print('Accuracy achieved with naive baseline approach: {:.2f}%'.format((naive_accuracy/len(gold))*100))
print("Accuracy of the Naive Bayes Classifier: {:.2f}%".format((nltk.classify.accuracy(
    classifier, test_set)*100)))
print('Accuracy achieved by guessing: {:.2f}%'.format((1/15)*100))

As we can see, our neural network works much, much better (around 50% accuracy) than any of the other three methods (around 15% accuracy for the naive baseline, around 16% accuracy for the Naive Bayes Classifier, and 6.67% accuracy by guessing). Unsurprisingly, the machine has a hard time understanding 'humor'. Ironically, also 'science_fiction' appears to be difficult to get right. The best recognized categories seem to be 'learned' and 'belles_lettres', although 'belles_lettres' also seems to cause much confusion: the algorithm sorts numerous snippets that belong to other categories into 'belles_lettres'. This boils down to the fact that even after excluding the most common words, many words still appear in all categories and 'belles_lettres' seems to have the highest frequencies for those words so that's what the neural network picks as the most likely category.

The categories that are the hardest and easiest to get right are respectively the two least and most frequent categories. It appears that despite our best efforts to normalize the word frequencies, it isn't enough. We would need more actual data to get better results. 

### Conclusion 

What can we learn from this project? Our neural network performs significantly better than the Naive Bayes Classifier although they aren't that different. They both look at the words in a text snippet and check if those words have appeared in a certain category before. The Naive Bayes Classifier gets back a simple yes or no answer, whereas our neural network will get back a more detailed answer: 'yes, 248 times', 'no, never', or 'yes, but only 3 times'. This comes back to the amount of data available: the more times you've come across some event in a particular setting, the more certain you can be that it is connected to that setting. However, with more data there will also be more noise that needs to be weeded out so it is not as simple as 'the more data, the better'. Yes, a large volume of data is a good start, but the more important factor is picking the right features from that data. 